# PIPELINE TO COMPARE ALGORITHMS
---
---

    def alg tal (image_refs, image_pbs, save_output_plots_path=None):
        ...
        return predicted_delta_phiCRs, times (de cada pairwise image pair ref pb -que están en los mismos indices claro)

Ke guarde los plots ke se outputean si hace falta en ese path dado.

Ta gero funkiño bat que coja algs, que coja image pairs y si acaso coja sus ground-truths como opctional argument, y que te outputee la tabla de imagen, algoritmos delta phicR, delta pol, times, GT, absolute errors.

Ta gero bebai outputee pa cada algoritmo un histograma de los absolute errors y un histograma de tiempos, con las medias y percentiles indicados correctamente.

In [4]:
import os
import json as json
import numpy as np
import pandas as pd
from SOURCE.CLASS_CODE_Polarization_Obtention_Algorithms import Rotation_Algorithm, Mirror_Flip_Algorithm
from SOURCE.CLASS_CODE_Image_Manager import Image_Manager
from SOURCE.CLASS_CODE_Ad_Hoc_Optimizer import Ad_Hoc_Optimizer
import matplotlib.pyplot as plt
import torch

assert torch.cuda.is_available(), "GPU is not enabled"
# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

def angle_to_pi_pi( angle): # convert any angle to range (-pi,pi]\n",
    angle= angle%(2*np.pi) # take it to [-2pi, 2pi]\n",
    return angle-np.sign(angle)*2*np.pi if abs(angle)>np.pi else angle

def run_benchmark_output_result_histograms_and_result_table( algorithm_lambda_list, algorithm_name_list,
                                            references, problems, image_pair_names,
                                            output_units='rad', ground_truths=None, GT_units=None,
                                            GT_nature = 'phiCR',
                                            experiment_name = None, output_path=None):
    # GTs should be in [-pi, pi] or [-180, 180]
    times = {}
    predicted_delta_phiCRs = {}
    conv = 180/np.pi if output_units=='deg' else 1
    convGT = 180/np.pi if (output_units=='deg' and GT_units=='rad') else \
        np.pi/180 if (output_units=='rad' and GT_units=='deg') else 1
    
    for algorithm, alg_name in zip(algorithm_lambda_list, algorithm_name_list):
        if output_path is not None:
            dir_for_alg = output_path+f"/{alg_name}/"
            os.makedirs( dir_for_alg, exist_ok=True )
        else:
            dir_for_alg = None
        predicted_delta_phiCRs[alg_name], times[alg_name] = algorithm(references, problems, dir_for_alg)
        
    json.dump({'image_pair_names':image_pair_names, 'predicted_delta_phiCRs':predicted_delta_phiCRs,
              'times':times}, open( f"{output_path}/RAW_results.json", "w"))
    # Rearrange the result to our desired Table and unit formats
    image_ids = []
    image_names = []
    algorithm_names = []
    delta_phiCRs = []
    delta_pols = []
    timess = []
    GTs = []
    abs_errors = []
    correct_decimals = []
    
    for idx, image_pair_name in enumerate(image_pair_names):
        image_ids.append(idx)
        image_names.append(idx)
        delta_phiCRs.append(idx)
        for algorithm, alg_name in zip(algorithm_lambda_list, algorithm_name_list):
            algorithm_names.append(alg_name)
            delta_phiCRs.append( conv*angle_to_pi_pi(predicted_delta_phiCRs[alg_name][idx]) ) 
            delta_pols.append( conv*angle_to_pi_pi(predicted_delta_phiCRs[alg_name][idx])/2.0 )
            times.append(times[alg_name][idx])
            if ground_truths is not None:
                GTs.append(convGT*ground_truths[idx])
                if GT_nature=='phiCR':
                    abs_errors.append( np.abs(delta_phiCRs[-1]-convGT*ground_truths[idx]) )
                else: # then GT is of polarization
                    abs_errors.append( np.abs(delta_pols[-1]-convGT*ground_truths[idx]) )
                correct_decimals.append() # beittu HISTOGRAMAGAZ batera zelan eitten zendun hau!
    table_per_image = pd.DataFrame.from_dict({'ID':image_ids, 'Image_Pair_Name':image_names, 'Algorithm':algorithm_names,
                                   'Predicted_Delta_PhiCRs':delta_phiCRs, 'Pred_Delta_Polarizt':delta_pols,
                                   'Times':timess, f'Ground_Truth_{GT_nature}':GTs, 'Absolute Error':abs_errors})
            
    # table_per_alg = # cuando copies el bootstrap para hacer els histograms ba kompleteu tabla hau
    # en prkipi deberia tener cols alg name, sample mean abs error, sample abs error stdv, sample abs error CI bootstrap
    # Y unos percentiles de los abs errors. Kiza lo mismo tb en términos de decimales correctos? Sí a cuanto porciento
    # de los samples se hizo con tantos decimales correctos, cuantos con tantos etc.    

/home/melanie/anaconda3/envs/fbvars/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/melanie/anaconda3/envs/fbvars/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Using device cuda:0


# Generic Routines

In [ ]:
def compute_intensity_gravity_center(image):
    """
        Expects input image to be an array of dimensions [h, w].
        It will return an array of gravity centers [2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to numpy indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = np.sum(image, axis=0) # weights for x [raw_width]
    intensity_in_h = np.sum(image, axis=1) # weights for y [raw_height]
    total_intensity = intensity_in_h.sum()

    # Compute mass center for intensity
    # [2] (h_center,w_center)
    return np.nan_to_num( np.stack(
        (np.dot(intensity_in_h, np.arange(image.shape[0]))/total_intensity,
         np.dot(intensity_in_w, np.arange(image.shape[1]))/total_intensity)
        ) )

def compute_raw_to_centered_iX(image, X):

    g_raw = compute_intensity_gravity_center(image)
    # crop the iamges with size (X+1+X)^2 leaving the gravity center in
    # the central pixel of the image. In case the image is not big enough for the cropping,
    # a 0 padding will be made.
    centered_image = np.zeros( (2*X+1, 2*X+1),  dtype = image.dtype )

    # we round the gravity centers to the nearest pixel indices
    g_index_raw = np.rint(g_raw).astype(int) #[N_images, 2]

    # obtain the slicing indices around the center of gravity
    # TODO -> make all this with a single array operation by stacking the lower and upper in
    # a new axis!!
    # [ 2 (h,w)]
    unclipped_lower = g_index_raw[:]-X
    unclipped_upper = g_index_raw[:]+X+1
    # unclippde could get out of bounds for the indices, so we clip them
    lower_bound = np.clip( unclipped_lower, a_min=0, a_max=image.shape)
    upper_bound = np.clip( unclipped_upper, a_min=0, a_max=image.shape)
    # we use the difference between the clipped and unclipped to get the necessary padding
    # such that the center of gravity is left still in the center of the image
    padding_lower = lower_bound-unclipped_lower
    padding_upper = upper_bound-unclipped_upper

    # crop the image
    centered_image[padding_lower[0]:padding_upper[0] or None,
                                    padding_lower[1]:padding_upper[1] or None ] = \
                  image[lower_bound[0]:upper_bound[0],
                                      lower_bound[1]:upper_bound[1]]
    return centered_image



def compute_intensity_gravity_centers_torch( images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=self.device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=self.device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)


def compute_raws_to_centered_iXs_torch( images, X, device):

    g_raw = compute_intensity_gravity_centers_torch(images) # [ N_images, 2]

    # crop the iamges with size (X+1+X)^2 leaving the gravity center in
    # the central pixel of the image. In case the image is not big enough for the cropping,
    # a 0 padding will be made.
    centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                  device=device)

    # we round the gravity centers to the nearest pixel indices
    g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

    # obtain the slicing indices around the center of gravity
    # TODO -> make all this with a single array operation by stacking the lower and upper in
    # a new axis!!
    # [ N_images, 2 (h,w)]
    unclipped_lower = g_index_raw-X
    unclipped_upper = g_index_raw+X+1

    # unclipped could get out of bounds for the indices, so we clip them
    lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                             max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(self.device)).int()
    upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                             max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
    # we use the difference between the clipped and unclipped to get the necessary padding
    # such that the center of gravity is left still in the center of the image
    padding_lower = lower_bound-unclipped_lower
    padding_upper = upper_bound-unclipped_upper

    # crop the image
    for im in range(g_raw.shape[0]):
        centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                    padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                  images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                      lower_bound[ im, 1]:upper_bound[ im, 1]]

    return centered_images

# PRE-PROCESSING FUNCTIONS

## Numpy in out

In [1]:
def normalize_to_max_saturate_and_iX(images,  saturation_threshold, dtype=np.float32,
                              iX_dev='cpu', out_dev='cpu', X=302): # threshold is in [0,1] of max
                                                              # images expected to be [N_images, h, w]
    images = images.astype(dtype)
    maxs = np.expand_dims( np.amax(images, axis=(-2,-1) ), (-2,-1) )
    images = np.where(images<maxs*saturation_threshold, images, 0.0)/maxs
    return compute_raws_to_centered_iXs_torch( torch.from_numpy(images).to(device), X, device).to('cpu').numpy()

def normalize_to_mean_saturate_to_max_and_iX(images,  saturation_threshold, dtype=np.float32,
                                     iX_dev='cpu', out_dev='cpu', X=302):  # threshold is in [0,1] of max
                                                                # images expected to be [N_images, h, w]
    images = images.astype(dtype)
    maxs = np.expand_dims( np.amax(images, axis=(-2,-1) ), (-2,-1) )
    images = np.where(images<maxs*saturation_threshold, images, 0.0)/np.expand_dims( np.mean(images, axis=(-2,-1) ), (-2,-1) )
    return compute_raws_to_centered_iXs_torch( torch.from_numpy(images).to(device), X, device).to('cpu').numpy()

def normalize_to_mean_saturate_to_mean_and_iX(images,  saturation_threshold, dtype=np.float32,
                                      iX_dev='cpu', out_dev='cpu', X=302):  # threshold is in [0,1] of max
                                                                # images expected to be [N_images, h, w]
    images = images.astype(dtype)
    means = np.expand_dims( np.mean(images, axis=(-2,-1) ), (-2,-1) )
    images = np.where(images<means*saturation_threshold, images, 0.0)/means
    return compute_raws_to_centered_iXs_torch( torch.from_numpy(images).to(device), X, device).to('cpu').numpy()

def normalize_to_max_and_iX(images, dtype=np.float32,
                    iX_dev='cpu', out_dev='cpu', X=302): # images expected to be [N_images, h, w]
    images= images.astype(dtype)/np.expand_dims( np.amax(images, axis=(-2,-1) ), (-2,-1) )
    return compute_raws_to_centered_iXs_torch( torch.from_numpy(images).to(device), X, device).to('cpu').numpy()

def normalize_to_mean_and_iX(images, dtype=np.float32,
                     iX_dev='cpu', out_dev='cpu', X=302): # images expected to be [N_images, h, w]
    images = images.astype(dtype)/np.expand_dims( np.mean(images, axis=(-2,-1) ), (-2,-1) )
    return compute_raws_to_centered_iXs_torch( torch.from_numpy(images).to(device), X, device).to('cpu').numpy()

## Torch in out

In [1]:
def normalize_to_max_saturate_and_iX_torch(images,  in_are_dev_float, saturation_threshold, 
                                           device, dtype=troch.float32, X=302): # threshold is in [0,1] of max
                                                              # images expected to be [N_images, h, w]
    if not in_are_dev_float:
        images = images.type(dtype).to(device)
    maxs = images.amax(dim=(-2,-1), keepdim=True)[0].unsqueeze(1)
    images = torch.where(images<maxs*saturation_threshold, images, 0.0)/maxs
    return compute_raws_to_centered_iXs_torch( images, X, device)

def normalize_to_mean_saturate_to_max_and_iX_torch(images,  in_are_dev_float, saturation_threshold, 
                                           device, dtype=troch.float32, X=302):  # threshold is in [0,1] of max
                                                                # images expected to be [N_images, h, w]
    if not in_are_dev_float:
        images = images.type(dtype).to(device)
    maxs = images.amax(dim=(-2,-1), keepdim=True)[0].unsqueeze(1)
    images = torch.where(images<maxs*saturation_threshold, images, 0.0)/torch.mean(images, axis=(-1,-2), keepdims=True)
    return compute_raws_to_centered_iXs_torch( images, X, device)


def normalize_to_mean_saturate_to_mean_and_iX_torch(images,  in_are_dev_float, saturation_threshold, 
                                           device, dtype=troch.float32, X=302):  # threshold is in [0,1] of max
                                                                # images expected to be [N_images, h, w]
    if not in_are_dev_float:
        images = images.type(dtype).to(device)
    means=torch.mean(images, axis=(-1,-2), keepdims=True)
    images = np.where(images<means*saturation_threshold, images, 0.0)/means
    return compute_raws_to_centered_iXs_torch( images, X, device)

def normalize_to_max_and_iX_torch(images, in_are_dev_float, 
                                device, dtype=troch.float32, X=302): # images expected to be [N_images, h, w]
    if not in_are_dev_float:
        images = images.type(dtype).to(device)
    return compute_raws_to_centered_iXs_torch(images/(images.amax(dim=(-2,-1), keepdim=True)[0].unsqueeze(1)), X, device)


def normalize_to_mean_and_iX_torch(images, in_are_dev_float, 
                                device, dtype=troch.float32, X=302): # images expected to be [N_images, h, w]
    if not in_are_dev_float:
        images = images.type(dtype).to(device)
    return compute_raws_to_centered_iXs_torch(images/torch.mean(images, axis=(-1,-2), keepdims=True), X, device)


# (A) ROTATION ALGORITHM

In [ ]:
rotation_alg_kw_args = {'theta_min_Rot':-np.pi, 'theta_max_Rot':np.pi, 'initial_guess_delta_rad':0.4, 
                        'use_exact_gravicenter':True, 'interpolation_flag':cv2.INTER_CUBIC}
rotation_fibo_kw_args = {'precision_quadratic':1e-10, 'max_it_quadratic':100, 'cost_tolerance_quadratic':1e-14}
rotation_quad_kw_args = {'precision_fibonacci':1e-10, 'max_it_fibonacci':100, 'cost_tolerance_fibonacci':1e-14}

rotation_preprocess_fct = lambda(images):  normalize_to_mean_and_iX(images)

# input images expected for all cases to be float64 and normalized to unity
# also, at least in this case, expected to be numpy arrays!
# Input expected to be alread [n, 2X+1, 2X+1] centered in gravicenter!
def run_rotation_algorithm(references, problems, image_pair_names, preprocess_fct, search_algorithm, 
                       search_alg_kw_args, rotation_alg_kw_args, out_plot_path=None, rotation_algorithm=None):
    predicted_deltaPhiCRs={}
    times={}
    
    image_loader = Image_Manager(mode=X, interpolation_flag=None)
    if rotation_algorithm is None:
        rotation_algorithm = Rotation_Algorithm(image_loader,
            rotation_alg_kw_args['theta_min_Rot'], rotation_alg_kw_args['theta_max_Rot'], None,
            rotation_alg_kw_args['initial_guess_delta_rad'], rotation_alg_kw_args['use_exact_gravicenter'], 
                                                initialize_it=False)
    image_names = []
    for mode in ['Ref', 'Pb']:
        for image_pair_name in image_pair_names:
            image_names.append(f"{mode}__{image_pair_name}")
    # charge the image loader:
    if preprocess_fct is not None:
        images = preprocess_fct( np.concatenate((references, problems), axis=0) )
    else:
        images = np.concatenate((references, problems), axis=0)
    image_loader.import_converted_images_as_array(images, image_names)
    # Execute the Rotation Algorithm:
    rotation_algorithm.interpolation_flag=rotation_alg_kw_args['interpolation_flag']
    rotation_algorithm.reInitialize(image_loader)
    # run it
    if search_algorithm=='quadratic':
        rotation_algorithm.quadratic_fit_search(search_alg_kw_args['precision_quadratic'], 
                            search_alg_kw_args['max_it_quadratic'], search_alg_kw_args['cost_tolerance_quadratic'])
    else: # 'fibo'
        rotation_algorithm.fibonacci_ratio_search(search_alg_kw_args['precision_fibonacci'],
                    search_alg_kw_args['max_points_fibonacci'], search_alg_kw_args['cost_tolerance_fibonacci'])    
    
    if out_plot_path is not None:
        rotation_algorithm.save_result_plots_fibonacci_or_quadratic(out_plot_path)
    
    for i in range(len(image_pair_names)):
        predicted_deltaPhiCRs.append( 
            rotation_algorithm.angles.values()[i+len(image_pair_names)] - \
                                      rotation_algorithm.angles.values()[i] ) # pb - ref
        times.append( rotation_algorithm.times.values()[i+len(image_pair_names)] + \
                                      rotation_algorithm.times.values()[i])
    
    return predicted_deltaPhiCRs, times

# (B) MIRROR FLIP ALGORITHM

In [ ]:
mirror_alg_kw_args = {'theta_min_Mir':0, 'theta_max_Mir':np.pi, 'initial_guess_delta_rad':0.4, 
                        'use_exact_gravicenter':True, 'interpolation_flag':cv2.INTER_CUBIC}
mirror_fibo_kw_args = {'precision_quadratic':1e-10, 'max_it_quadratic':100, 'cost_tolerance_quadratic':1e-14}
mirror_quad_kw_args = {'precision_fibonacci':1e-10, 'max_it_fibonacci':100, 'cost_tolerance_fibonacci':1e-14}

mirror_preprocess_fct = lambda images: normalize_to_mean_and_iX(images)
# input images expected for all cases to be float64 and normalized to unity
# also, at least in this case, expected to be numpy arrays!
# Input expected to be alread [n, 2X+1, 2X+1] centered in gravicenter!
def run_mirror_flip_algorithm(references, problems, image_pair_names, preprocess_fct, search_algorithm, 
                       search_alg_kw_args, mirror_alg_kw_args, out_plot_path=None, mirror_algorithm=None):
    predicted_deltaPhiCRs={}
    times={}
    
    image_loader = Image_Manager(mode=X, interpolation_flag=None)
    if mirror_algorithm is None:
        mirror_algorithm = Mirror_Flip_Algorithm(image_loader,
            mirror_alg_kw_args['theta_min_Mir'], mirror_alg_kw_args['theta_max_Mir'], None,
            mirror_alg_kw_args['initial_guess_delta_rad'], method="aff", left_vs_right=True, 
            use_exact_gravicenter=mirror_alg_kw_args['use_exact_gravicenter'], initialize_it=False)

    image_names = []
    for mode in ['Ref', 'Pb']:
        for image_pair_name in image_pair_names:
            image_names.append(f"{mode}__{image_pair_name}")
    # charge the image loader:
    if preprocess_fct is not None:
        images = preprocess_fct( np.concatenate((references, problems), axis=0) )
    else:
        images = np.concatenate((references, problems), axis=0)
    image_loader.import_converted_images_as_array(images, image_names)
    # Execute the Rotation Algorithm:
    mirror_algorithm.interpolation_flag=mirror_alg_kw_args['interpolation_flag']
    mirror_algorithm.reInitialize(image_loader)
    # run it
    if search_algorithm=='quadratic':
        mirror_algorithm.quadratic_fit_search(search_alg_kw_args['precision_quadratic'], 
                            search_alg_kw_args['max_it_quadratic'], search_alg_kw_args['cost_tolerance_quadratic'])
    else: # 'fibo'
        mirror_algorithm.fibonacci_ratio_search(search_alg_kw_args['precision_fibonacci'],
                    search_alg_kw_args['max_points_fibonacci'], search_alg_kw_args['cost_tolerance_fibonacci'])    
    
    if out_plot_path is not None:
        mirror_algorithm.save_result_plots_fibonacci_or_quadratic(out_plot_path)
    
    for i in range(len(image_pair_names)):
        predicted_deltaPhiCRs.append( 
            mirror_algorithm.angles.values()[i+len(image_pair_names)] - \
                                      mirror_algorithm.angles.values()[i] ) # pb - ref
        times.append( mirror_algorithm.times.values()[i+len(image_pair_names)] + \
                                      mirror_algorithm.times.values()[i])
    
    return predicted_deltaPhiCRs, times

# (C) Simulation Coordinate Descent using h5f library of D matrices

In [ ]:
class Simulation_precomputed_R0_w0_Z_Dmatrices_optimize_phiCR():
    def __init__(self,  ID_file_path, D_matrix_file_path, device, coordinate_descent_cycles,
                 min_angle, max_angle, initial_guess_delta_rad, initial_guess_delta_R0,ñ
                 preprocess_fct, similarity, R0_init_guess, w0_init_guess, Z_init_guess):
        import json
        self.D_matrix_file_path = D_matrix_file_path
        self.df_GTs = pd.DataFrame.from_dict(json.load(open(ID_file_path)))       
        self.R0s = list(self.df_GTs['R0s'].drop_duplicates()) # Note they are lists of strings!
        self.w0s = list(self.df_GTs['w0s'].drop_duplicates())
        self.Zs = list(self.df_GTs['Zs'].drop_duplicates())
        self.R0s_ar = np.array(self.R0s, dtype=np.float64) # Convert them to float arrays
        self.w0s_ar = np.array(self.w0s, dtype=np.float64)
        self.Zs_ar = np.array(self.Z0s, dtype=np.float64)
        self.device = device
        if not hasattr(self, 'h5f_D_matrices'): # this was originally thought for multiprocess child threads to have the chance each to have an open copy of the hdf5 (which should be opened in the first iteration of the parallelized get, not here in the init, but well...)
            self.open_hdf5()
            self.phis = torch.from_numpy(self.h5f_D_matrices['phis'][:]).unsqueeze(0).to(self.device) #[1,Nx,Ny]

        # search function params?
        self.cycles=coordinate_descent_cycles
        self.min_Z=min(self.Zs)
        self.max_Z=max(self.Zs)
        self.min_phi=min_angle
        self.max_phi=max_angle
        self.min_radi=min(self.R0s)
        self.max_radi=max(self.R0s)
        self.min_w0=min(self.w0s)
        self.max_w0=max(self.w0s)
        self.initial_guess_delta_rad=initial_guess_delta_rad
        self.initial_guess_delta_R0=initial_guess_delta_R0
        
        self.R0_precision=abs(self.R0s[1]-self.R0s[0])
        self.w0_precision=abs(self.w0s[1]-self.w0s[0])
        self.Z_precision=abs(self.Zs[1]-self.Zs[0])
        # pa phiCR no porke es unbounded!
        self.optimizer = Ad_Hoc_Optimizer(min_angle, max_angle, initial_guess_delta, self.evaluate_image_closest_angle)
        self.choose_similarity_alg(similarity)
        
        self.times={}
        self.radious_points={}
        self.w0_points={}
        self.Z_points={}
        self.phi_points={}
        # use the results from the gradient algorithm to initialize the best triplet?
        self.R0_pix_best={}
        self.w0_best={}
        self.Z_best={}
        self.phi_CR_best={}
        self.last_cycle=coordinate_descent_cycles

        self.best_radii={}
        self.best_w0s={}
        self.best_zs={}
        self.best_angles={}
        self.simulations_required={}
        
        self.similarity=similarity # the similarity metric to use as the objective function
        self.preprocess_fct = preprocess_fct
        self.last_R0=None
        self.last_w0=None
        self.last_Z=None
        
        self.R0_init_guess =R0_init_guess
        self.w0_init_guess = w0_init_guess
        self.Z_init_guess = Z_init_guess
        
        image_loader = Image_Manager(mode=X, interpolation_flag=None)
        self.algorithm_M = Mirror_Flip_Algorithm(image_loader,
            mirror_alg_kw_args['theta_min_Mir'], mirror_alg_kw_args['theta_max_Mir'], None,
            mirror_alg_kw_args['initial_guess_delta_rad'], method="aff", left_vs_right=True, 
            use_exact_gravicenter=mirror_alg_kw_args['use_exact_gravicenter'], initialize_it=False)
        
    def __del__(self):
        if hasattr(self, 'h5f_D_matrices'):
            self.h5f_D_matrices.close()
            
    def open_hdf5(self):
        self.h5f_D_matrices = h5py.File( self.D_matrix_file_path, 'r')
        #self.dataset = self.img_hdf5['dataset'] # if you want dataset.

    def angle_to_pi_pi(self, angle): # convert any angle to range ()-pi,pi]
        angle= angle%(2*np.pi) # take it to [-2pi, 2pi]
        return angle-np.sign(angle)*2*np.pi if abs(angle)>np.pi else angle
        
    def closest_idx_in_ar(self, value, list_array):
        return (np.abs(list_array-value)).argmin()
        
    def get_simulated_image(self, R0, w0, Z, phiCR):
        if self.last_R0!=R0 or self.last_w0!=w0 or self.last_Z!=Z:
            self.D_mats = torch.from_numpy(self.h5f_D_matrices[
                f"R0_{self.closest_idx_in_ar(R0, self.R0s_ar)}_w0_{self.closest_idx_in_ar(w0, self.w0s_ar)}_Z_{self.closest_idx_in_ar(Z0, self.Z0s)}"][:]
                                 ).unsqueeze(1).to(self.device) #[2, 1, h, w]            
            self.last_R0 = R0
            self.last_w0 = w0
            self.last_Z = Z
        phiCRs = torch.FloatTensor(self.angle_to_pi_pi(phiCR)).to(self.device) #[num_phiCR, 1, 1]
        images = self.D_mats[0]+self.D_mats[1]*torch.cos(phiCRs-self.phis) #[num_phiCR, Nx,Ny]
        return self.preprocess_fct(images, in_are_dev_float=True, device=self.device)

    def choose_similarity_alg(self, similarity):
        if similarity == 'mse': # mean square error
            self.similarity_func = lambda im1, im2: torch.sum(torch.square(im1-im2))
        elif similarity == 'mae': # mean absolute error
            self.similarity_func = lambda im1, im2: torch.sum(np.abs(im1-im2))
        # NOT TORCHIZISED!
        elif similarity == 'ssim':
            self.similarity_func = lambda im1, im2: -ism.ssim(im1, im2)
        elif similarity == 'fsim':
            self.similarity_func = lambda im1, im2: -ism.fsim(np.expand_dims(im1,2), np.expand_dims(im2,2))
        elif similarity == 'issm':
            self.similarity_func = lambda im1, im2: ism.issm(im1, im2)
        elif similarity == 'sre':
            self.similarity_func = lambda im1, im2: -ism.sre(np.expand_dims(im1,2), np.expand_dims(im2,2))
        elif similarity == 'sam':
            self.similarity_func = lambda im1, im2: -ism.sam(np.expand_dims(im1,2), np.expand_dims(im2,2))
        elif similarity == 'uiq':
            self.similarity_func = lambda im1, im2: -ism.uiq(np.expand_dims(im1,2), np.expand_dims(im2,2))
        # ADD DL trained Similarity Metric!

        
    def reInitialize_and_input_images(self, images, images_names,
                     preprocess_fct=None, coordinate_descent_cycles=None, similarity=None,
                    R0_init_guess, w0_init_guess, Z_init_guess):       
        self.images = self.preprocess_fct(images, in_are_dev_float=False, device=self.device)
        self.image_names = image_names
        self.times={}
        self.radious_points={}
        self.w0_points={}
        self.Z_points={}
        self.phi_points={}
        self.R0_pix_best={}
        self.w0_best={}
        self.Z_best={}
        self.phi_CR_best={}
        self.last_cycle=coordinate_descent_cycles

        self.best_radii={}
        self.best_w0s={}
        self.best_zs={}
        self.best_angles={}
        self.simulations_required={}
        
        self.last_R0=None
        self.last_w0=None
        self.last_Z=None

        image_loader = Image_Manager(mode=X, interpolation_flag=None)
        self.image_loader_for_M.import_converted_images_as_array( self.images.to('cpu').numpy(), self.image_names )
        self.algorithm_M.reInitialize(image_loader_for_M)
        
        if similarity is not None:
            self.similarity=similarity
            self.choose_similarity_alg(similarity)
        if preprocess_fct is not None:
            self.preprocess_fct = preprocess_fct
        if coordinate_descent_cycles is not None:
            self.cycles=coordinate_descent_cycles
        if R0_init_guess is not None:
            self.R0_init_guess = R0_init_guess
        if w0_init_guess is not None:
            self.w0_init_guess = w0_init_guess
        if Z_init_guess is not None:
            self.Z_init_guess = Z_init_guess

    def evaluate_simulation_phi(self, exp_image, angle, R0_best, Z_best, w0_best): # exp im expected to be [1,h,w]
        return self.similarity(exp_image, self.get_simulated_image(R0_best, w0_best, Z_best, phiCR))

    def evaluate_simulation_R0(self, exp_image, R0, angle_best, Z_best, w0_best):
        return self.similarity(exp_image, self.get_simulated_image(R0, w0_best, Z_best, angle_best))


    def evaluate_simulation_Z(self, exp_image, Z, angle_best, R0_best, w0_best):
        return self.similarity(exp_image, self.get_simulated_image(R0_best, w0_best, Z, angle_best))


    def evaluate_simulation_w0(self, exp_image, w0, angle_best, R0_best, Z_best):
        return self.similarity(exp_image, self.get_simulated_image(R0_best, w0, Z_best, angle_best))


    def fibonacci_ratio_search(self, precision_pix, maximum_points_R0, precision_phi,
        maximum_points_phi, precision_Z, maximum_points_Z,  precision_w0, maximum_points_w0, cost_tol,
        precision_M, maximum_points_M, cost_tol_M):
        # We first find an estimation for phiCR using the mirror flip algorithm
        self.algorithm_M.fibonacci_ratio_search(precision_M, maximum_points_M, cost_tol_M)
        
        # We now execute a sequence of linear optimizations for each coordinate for every cycle and image
        for im, name in enumerate(self.image_names):
            self.times[name]={}
            self.radious_points[name]={}
            self.Z_points[name]={}
            self.w0_points[name]={}
            self.phi_points[name]={}
            # use the results from the gradient algorithm to initialize the best triplet
            #self.R0_pix_best[name]=[1.65*abs(self.algorithm_G.optimals[f'Fibonacci_Search_{name}'])]
            self.R0_pix_best[name]=[self.R0_init_guess]
            self.Z_best[name]=[self.Z_init_guess]
            self.w0_best[name]=[self.w0_init_guess]
            # Initialize the phiCR with the result from the mirror flip agorithm
            self.phi_CR_best[name]=[self.algorithm_M.angles[f'Fibonacci_Search_{name}']]
            simulations=0
            for cycle in range(self.cycles):
                # Optimize Radious magnitude - the width of the croissant
                (time_w0, self.w0_points[name][cycle], w0_best, w0_optimum,
                 _)=self.w0_optimizer.fibonacci_ratio_search(
                    precision_w0, maximum_points_w0, cost_tol,
                    self.images_normFloat[im],
                    (self.phi_CR_best[name][-1], self.R0_pix_best[name][-1], self.Z_best[name][-1]), self.w0_best[name][-1]
                )

                # Optimize Radious
                (time_rad, self.radious_points[name][cycle], R0_pix_best, radi_optimum,
                 _)=self.radious_optimizer.fibonacci_ratio_search(
                    precision_pix, maximum_points_R0, cost_tol,
                    self.images[im],
                    (self.phi_CR_best[name][-1], self.Z_best[name][-1], w0_best), self.R0_pix_best[name][-1]
                )

                # Optimize Angle
                (time_ang, self.phi_points[name][cycle], phi_CR_best, phi_optimum, _)=self.angle_optimizer.fibonacci_ratio_search(
                    precision_phi, maximum_points_phi, cost_tol,
                    self.images[im],
                    (R0_pix_best, self.Z_best[name][-1], w0_best), self.phi_CR_best[name][-1]
                )

                # Optimize Z
                if maximum_points_Z!=0:
                    self.Z_optimizer.a=-2*np.sqrt(1/3)*w0_best
                    self.Z_optimizer.b=-self.Z_optimizer.a
                    (time_Z, self.Z_points[name][cycle], Z_best, Z_optimum,
                        _)=self.Z_optimizer.fibonacci_ratio_search(
                            precision_Z, maximum_points_Z, cost_tol,
                            self.images[im],
                            (phi_CR_best, R0_pix_best, w0_best), self.Z_best[name][-1]
                        )
                else: # dont optimize z
                    time_Z=0
                    Z_best=0
                    Z_optimum=0
                    self.Z_points[name][cycle]=np.array([[0,0,0]])


                self.times[name][cycle]=time_rad+time_ang+time_Z+time_w0
                self.R0_pix_best[name].append(R0_pix_best)
                self.phi_CR_best[name].append(phi_CR_best)
                self.w0_best[name].append(w0_best)
                self.Z_best[name].append(Z_best)

                simulations+=len(self.phi_points[name][cycle])+len(self.radious_points[name][cycle])+len(self.Z_points[name][cycle])+len(self.w0_points[name][cycle])

                # check if convergence criterion is met
                if((abs(Z_optimum-phi_optimum)<cost_tol and
                        abs(phi_optimum-radi_optimum)<cost_tol and
                            abs(radi_optimum-w0_optimum)<cost_tol)
                        or (abs(self.R0_pix_best[name][-2]-R0_pix_best)<precision_pix and
                        abs(self.phi_CR_best[name][-2]-phi_CR_best)<precision_phi and
                        abs(self.Z_best[name][-2]-Z_best)<precision_Z) and
                        abs(self.w0_best[name][-2]-w0_best<precision_w0)):
                    self.last_cycle=cycle+1
                    break
            self.best_radii[name]=R0_pix_best
            self.best_w0s[name]=w0_best
            self.best_zs[name]=Z_best
            self.best_angles[name]=self.angle_to_pi_pi(phi_CR_best)
            self.simulations_required[name]=simulations
            print(f"Image {im} optimized!")


    def quadratic_fit_search(self, precision_pix, maximum_points_R0, precision_phi,
        maximum_points_phi, precision_Z, maximum_points_Z, precision_w0, maximum_points_w0, cost_tol,
        precision_M, maximum_points_M, cost_tol_M):
        # We first find an estimation for R0 and phiCR using the gradient algorithm
        self.algorithm_M.quadratic_fit_search(precision_M, maximum_points_M, cost_tol_M)

        # We now execute a sequence of linear optimizations for each coordinate for every cycle
        for im, name in enumerate(self.image_names):
            self.times[name]={}
            self.radious_points[name]={}
            self.Z_points[name]={}
            self.w0_points[name]={}
            self.phi_points[name]={}
            # use the results from the gradient algorithm to initialize the best triplet
            #self.R0_pix_best[name]=[1.65*abs(self.algorithm_G.optimals[f'Fibonacci_Search_{name}'])]
            self.R0_pix_best[name]=[self.R0_init_guess]
            self.Z_best[name]=[self.Z_init_guess]
            self.w0_best[name]=[self.w0_init_guess]
            # Initialize the phiCR with the result from the mirror flip agorithm
            self.phi_CR_best[name]=[self.algorithm_M.angles[f'Fibonacci_Search_{name}']]
            simulations=0
            for cycle in range(self.cycles):
                # Optimize Radious magnitude - the width of the croissant
                (time_w0, self.w0_points[name][cycle], w0_best, w0_optimum,
                 _)=self.w0_optimizer.fibonacci_ratio_search(
                    precision_w0, maximum_points_w0, cost_tol,
                    self.images[im],
                    (self.phi_CR_best[name][-1], self.R0_pix_best[name][-1], self.Z_best[name][-1]), self.w0_best[name][-1]
                )

                # Optimize Radious
                (time_rad, self.radious_points[name][cycle], R0_pix_best, radi_optimum,
                 _)=self.radious_optimizer.quadratic_fit_search(
                    precision_pix, maximum_points_R0, cost_tol,
                    self.images[im],
                    (self.phi_CR_best[name][-1], self.Z_best[name][-1], w0_best), self.R0_pix_best[name][-1]
                )

                # Optimize Angle
                (time_ang, self.phi_points[name][cycle], phi_CR_best, phi_optimum, _)=self.angle_optimizer.quadratic_fit_search(
                    precision_phi, maximum_points_phi, cost_tol,
                    self.images[im],
                    (R0_pix_best, self.Z_best[name][-1], w0_best), self.phi_CR_best[name][-1]
                )

                # Optimize Z
                if maximum_points_Z!=0:
                    self.Z_optimizer.a=-2*np.sqrt(1/3)*R0_pix_best*self.simulator.dx
                    self.Z_optimizer.b=-self.Z_optimizer.a
                    (time_Z, self.Z_points[name][cycle], Z_best, Z_optimum,
                        _)=self.Z_optimizer.quadratic_fit_search(
                            precision_Z, maximum_points_Z, cost_tol,
                            self.images[im],
                            (phi_CR_best, R0_pix_best, w0_best), self.Z_best[name][-1]
                        )
                else: # dont optimize z
                    time_Z=0
                    Z_best=0
                    Z_optimum=0
                    self.Z_points[name][cycle]=np.array([[0,0,0]])

                self.times[name][cycle]=time_rad+time_ang+time_Z+time_w0
                self.R0_pix_best[name].append(R0_pix_best)
                self.w0_best[name].append(w0_best)
                self.phi_CR_best[name].append(phi_CR_best)
                self.Z_best[name].append(Z_best)

                simulations+=len(self.phi_points[name][cycle])+len(self.radious_points[name][cycle])+len(self.Z_points[name][cycle])+len(self.w0_points[name][cycle])

                # check if convergence criterion is met
                if((abs(Z_optimum-phi_optimum)<cost_tol and
                        abs(phi_optimum-radi_optimum)<cost_tol and
                            abs(radi_optimum-w0_optimum)<cost_tol)
                        or (abs(self.R0_pix_best[name][-2]-R0_pix_best)<precision_pix and
                        abs(self.phi_CR_best[name][-2]-phi_CR_best)<precision_phi and
                        abs(self.Z_best[name][-2]-Z_best)<precision_Z) and
                        abs(self.w0_best[name][-2]-w0_best<precision_w0)):
                    self.last_cycle=cycle+1
                    break
            self.best_radii[name]=R0_pix_best
            self.best_w0s[name]=w0_best
            self.best_zs[name]=Z_best
            self.best_angles[name]=self.angle_to_pi_pi(phi_CR_best)
            self.simulations_required[name]=simulations
            print(f"Image {im} optimized!")

    def plot_best_found_ones(self, out_path):
        os.makedirs(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/", exist_ok=True)
        fig = plt.figure(figsize=(2*4.5, len(self.image_names)*4.5))
        axes=fig.subplots(len(self.image_names),2)
        if len(axes.shape)==1:
            axes=np.expand_dims(axes, 0)
        for k, (name, optimal_angle) in enumerate(self.best_angles.items()):
            axes[k, 0].imshow(self.images[k].to('cpu').numpy())        
            axes[k, 1].imshow(self.get_simulated_image( R0=self.best_radii[name],
                w0=self.best_w0s[name], Z=self.best_Zs[name], phiCR=self.best_angles[name]).to('cpu').numpy())
            axes[k,0].set_title(f"Experimental Image {name}\nOptimal phiCR {optimal_angle}")
            axes[k,1].set_title(f"Fitted Image: \nR0={self.best_radii[name]}pix w0={self.best_w0s[name]}pix\n Z={self.best_Zs[name]}pix phiCR={self.best_angles[name]}rad")
        fig.suptitle("Results of the Simulation Coordinate Descent Algorithm")
        plt.savefig(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/Net_Results.png")
        
    def save_result_plots(self, out_path, meth_name):
        os.makedirs(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/", exist_ok=True)
        self.algorithm_M.save_result_plots_fibonacci_or_quadratic(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/")
        fig = plt.figure(figsize=(4*10, 10*self.last_cycle))

        for k, name in enumerate(self.phi_points.keys()):
            axes = fig.subplots(self.last_cycle, 4)
            if self.last_cycle==1:
                axes=np.expand_dims(axes,0)
            for cycle in range(self.last_cycle):
                axes[cycle, 0].plot(self.w0_points[name][cycle][:,0], self.w0_points[name][cycle][:,1], 'o', label=f'w0 pixels descent fixing phiCR={self.phi_CR_best[name][cycle]}; R0 pix={self.R0_pix_best[name][cycle]}; Z={self.Z_best[name][cycle]}')
                axes[cycle, 1].plot(self.radious_points[name][cycle][:,0], self.radious_points[name][cycle][:,1], 'o', label=f'R0 pixels descent fixing w0 pix ={self.w0_best[name][cycle+1]}; phiCR={self.phi_CR_best[name][cycle]}; Z={self.Z_best[name][cycle]}')
                axes[cycle, 2].plot(self.phi_points[name][cycle][:,0], self.phi_points[name][cycle][:,1], 'o', label=f'phiCR descent fixing w0 pix ={self.w0_best[name][cycle+1]}; R0 pix={self.R0_pix_best[name][cycle+1]}; Z={self.Z_best[name][cycle]}')
                axes[cycle, 3].plot(self.Z_points[name][cycle][:,0], self.Z_points[name][cycle][:,1], 'o', label=f'Z descent fixing w0 pix ={self.w0_best[name][cycle+1]}; R0 pix={self.R0_pix_best[name][cycle+1]}; phiCR={self.phi_CR_best[name][cycle+1]}')

                axes[cycle, 0].set_xlabel("w0 (pixels)")
                axes[cycle, 1].set_xlabel("R0 (pixels)")
                axes[cycle, 2].set_xlabel("phi_CR (rad)")
                axes[cycle, 3].set_xlabel("Z (w0-s)")
                axes[cycle, 0].set_title(f"Best of cycle={self.w0_best[name][cycle+1]}\n Computed points={len(self.w0_points[name][cycle])}")
                axes[cycle, 1].set_title(f"Best of cycle={self.R0_pix_best[name][cycle+1]}\n Computed points={len(self.radious_points[name][cycle])}")
                axes[cycle, 2].set_title(f"Best of cycle={self.phi_CR_best[name][cycle+1]}\n Computed points={len(self.phi_points[name][cycle])}")
                axes[cycle, 3].set_title(f"Best of cycle={self.Z_best[name][cycle+1]}\n Computed points={len(self.Z_points[name][cycle])}")

                for i in range(4):
                    axes[cycle, i].set_ylabel("sum(abs(simulated_image(phiCR, R0_pix, w0_pix, Z)-exp_image))")
                    axes[cycle, i].grid(True)
                    axes[cycle, i].legend()

                fig.suptitle(f"{meth_name}  {name}\nBest triplet: phiCR={self.best_angles[name]}rad; R0_pix={self.best_radii[name]}pix; w0={self.best_w0s[name]}(pix); Z={self.best_zs[name]}w0")
            plt.savefig(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/{meth_name}__{name}.png")
            fig.clf()
            I=self.get_simulated_image( R0=self.best_radii[name],
                w0=self.best_w0s[name], Z=self.best_Zs[name], phiCR=self.best_angles[name]).to('cpu').numpy()
            I = (255*I/I.max())[0] #[h,w]
            cv2.imwrite(f"{out_path}/Simulation_Coordinate_Descent_Algorithm/iX_{X}_Left_Exp_[{name}]_Right_Simul_PolAngle_{self.best_angles[name]/2:.15f}_CRAngle_{self.best_angles[name]:.15f}_Z_{self.best_zs[name]}_R0_pix_{self.best_radii[name]}_w0_pix_{self.best_w0s[name]}.png",
                np.concatenate((I, self.images[k]/self.images[k].max()),axis=1))

In [ ]:
ID_file_path= "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/STRUCTURE_Grid_R0_70_w0_70_Z_4.json"
D_matrix_file_path= "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Encoder_Alone/Dataset_R0_70_w0_70_Z_4.h5"

simulation_alg_kw_args = {'ID_file_path':ID_file_path, 'D_matrix_file_path':D_matrix_file_path, 'device':device, 
                        'coordinate_descent_cycles':2, 'min_angle':-np.pi,
                       'max_angle':np.pi,'initial_guess_delta_rad':0.1, 'initial_guess_delta_R0':1,
                        'similarity':'mae','R0_init_guess':158.0, 'w0_init_guess':25.0, 'Z_init_guess':0.0
                       }
simulation_fibo_kw_args = {'precision_pix':1e-5, 'maximum_points_R0':10, 'precision_phi':1e-8,
                        'maximum_points_phi':100, 'precision_Z':1e-2, 'maximum_points_Z':0, 
                         'precision_w0':1e-5, 'maximum_points_w0':10, 'cost_tol':1e-14, 'precision_M':1e-4,
                        'maximum_points_M':100, 'cost_tol_M':1e-14}

simulation_quad_kw_args = {'precision_pix':1e-5, 'maximum_points_R0':10, 'precision_phi':1e-8,
                        'maximum_points_phi':100, 'precision_Z':1e-2, 'maximum_points_Z':0, 
                         'precision_w0':1e-5, 'maximum_points_w0':10, 'cost_tol':1e-14, 'precision_M':1e-4,
                        'maximum_points_M':100, 'cost_tol_M':1e-14}

simulation_preprocess_fct = lambda(images, in_are_dev_float, device): normalize_to_mean_saturate_to_max_and_iX_torch(
                images,  in_are_dev_float,  device=device, saturation_threshold=0.1, dtype=troch.float32, X=302)

# also, at least in this case, expected to be torch arrays!
def run_simulation_algorithm(references, problems, image_pair_names, preprocess_fct, search_algorithm, 
                       search_alg_kw_args, rotation_alg_kw_args, out_plot_path=None, simulation_algorithm=None):
    predicted_deltaPhiCRs={}
    times={}
    
    if simulation_algorithm is None:
        simulation_algorithm = Simulation_precomputed_R0_w0_Z_Dmatrices_optimize_phiCR(
            ID_file_path=simulation_alg_kw_args['ID_file_path'], 
            D_matrix_file_path=simulation_alg_kw_args['D_matrix_file_path'], device=simulation_alg_kw_args['device'],
            coordinate_descent_cycles=simulation_alg_kw_args['coordinate_descent_cycles'],
            min_angle=simulation_alg_kw_args['min_angle'], max_angle=simulation_alg_kw_args['max_angle'], 
            initial_guess_delta_rad=simulation_alg_kw_args['initial_guess_delta_rad'], 
            initial_guess_delta_R0=simulation_alg_kw_args['initial_guess_delta_R0'],
            preprocess_fct=preprocess_fct, similarity=simulation_alg_kw_args['similarity'],
            R0_init_guess=simulation_alg_kw_args['R0_init_guess'], w0_init_guess=simulation_alg_kw_args['w0_init_guess'], 
            Z_init_guess=simulation_alg_kw_args['Z_init_guess'] 
            )
            
    image_names = []
    for mode in ['Ref', 'Pb']:
        for image_pair_name in image_pair_names:
            image_names.append(f"{mode}__{image_pair_name}")
    # charge the image loader:
    images = torch.cat((references, problems), dim=0)
    # Execute the Rotation Algorithm:
    simulation_algorithm.reInitialize_and_input_images( images, image_names,
                     preprocess_fct=preprocess_fct, coordinate_descent_cycles=simulation_alg_kw_args['coordinate_descent_cycles'],
                    similarity=simulation_alg_kw_args['similarity'],
                R0_init_guess=simulation_alg_kw_args['R0_init_guess'],
                w0_init_guess=simulation_alg_kw_args['w0_init_guess'],
                Z_init_guess=simulation_alg_kw_args['Z_init_guess'])
    # run it
    if search_algorithm=='quadratic':
        simulation_algorithm.quadratic_fit_search(precision_pix=search_alg_kw_args['precision_pix'],
            maximum_points_R0=search_alg_kw_args['maximum_points_R0'], precision_phi=search_alg_kw_args['precision_phi'],
            maximum_points_phi=search_alg_kw_args['maximum_points_phi'], precision_Z=search_alg_kw_args['precision_Z'], 
            maximum_points_Z=search_alg_kw_args['maximum_points_Z'],  precision_w0=search_alg_kw_args['precision_w0'],
            maximum_points_w0=search_alg_kw_args['maximum_points_w0'], cost_tol=search_alg_kw_args['cost_tol'],
            precision_M=search_alg_kw_args['precision_M'], maximum_points_M=search_alg_kw_args['maximum_points_M'],
                                                  cost_tol_M=search_alg_kw_args['cost_tol_M'])
        if out_plot_path is not None:
            rotation_algorithm.save_result_plots(out_plot_path, 'Quadratic')
    else: # 'fibo'
        simulation_algorithm.fibonacci_ratio_search(precision_pix=search_alg_kw_args['precision_pix'],
            maximum_points_R0=search_alg_kw_args['maximum_points_R0'], precision_phi=search_alg_kw_args['precision_phi'],
            maximum_points_phi=search_alg_kw_args['maximum_points_phi'], precision_Z=search_alg_kw_args['precision_Z'], 
            maximum_points_Z=search_alg_kw_args['maximum_points_Z'],  precision_w0=search_alg_kw_args['precision_w0'],
            maximum_points_w0=search_alg_kw_args['maximum_points_w0'], cost_tol=search_alg_kw_args['cost_tol'],
            precision_M=search_alg_kw_args['precision_M'], maximum_points_M=search_alg_kw_args['maximum_points_M'],
                                                  cost_tol_M=search_alg_kw_args['cost_tol_M'])    
        if out_plot_path is not None:
            rotation_algorithm.save_result_plots(out_plot_path, 'Fibonacci')

    if out_plot_path is not None:
        simulation_algorithm.plot_best_found_ones(out_plot_path)

    
    for i in range(len(image_pair_names)):
        predicted_deltaPhiCRs.append( 
            simulation_algorithm.best_angles.values()[i+len(image_pair_names)] - \
                                      simulation_algorithm.best_angles.values()[i] ) # pb - ref
        times.append( sum(simulation_algorithm.times.values()[i+len(image_pair_names)]) + \
                                      sum(simulation_algorithm.times.values()[i]))
    
    return predicted_deltaPhiCRs, times

# (D) Naive Affine Rotation

# (E) Fourier Space shift Rotation and scaling

# (F) CNN

# BENCHMARK

### (a) Using the Experimental+Simulated Test Images

In [ ]:
im_type=np.uint16 if image_depth==16 else np.uint8
max_intensity=65535 if image_depth==16 else 255
np.random.seed(randomization_seed)

### (b) Using a large simulated Noisy Image Library